In [1]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder, OneHotEncoder
from datetime import datetime



In [2]:
# --데이터 전처리
# 업체규모(EMP_CNT → 결측값 존재)
# 업력(ESTB_DATE → 결측값 존재)
# 매출액 – 상승추세
# 상장,비상장
# 휴폐업 이력 존재 유무

# 외부데이터
# 산업동향 – 산업위험
# ->통계청 업종별 경기 전망
# 기업 입지 - 도심지역, 외곽지역 (bizno 사업자등록번호 조회)
# (시장위험 – 경기,금리,환율,유가)

#KDD 데이터 분석 방법론 참고

# 데이터 가져오기

In [3]:
wb1 = openpyxl.load_workbook('./numble_data/휴폐업 중소법인 재무보유.xlsx')
wb2 = openpyxl.load_workbook('./numble_data/액티브 중소법인 재무보유.xlsx', read_only=True)

In [4]:
sheet_rest = wb1["기업개요"]
sheet_rest_record=wb1['휴폐업이력']

In [5]:
#휴폐업 - 기업 개요 (6739개)
df_rest = pd.DataFrame(sheet_rest.values)
df_rest.columns = df_rest.iloc[0,:]
df_rest = df_rest.iloc[1:, :]
df_rest = df_rest.replace('','NaN')
df_rest = df_rest.fillna('NaN')
df_rest = df_rest.reset_index(drop=True)
df_rest

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1268145633,(주),용진테크,NaN,"Yongjin Tech Co., Ltd.",M,2,2,1,1,...,N,NaN,20000529,01,NaN,자동차부품 제조판매업,NaN,www.yongjintech.com,안상동,40
1,1388101749,NaN,신구건설,(주),"SHINKU C0NSTRUCTION CO.,LTD.",M,2,2,1,1,...,N,NaN,19850110,01,NaN,건축자재 판매업,NaN,www.shinku.co.kr,소재필,13
2,1068179240,(주),석정미디어,NaN,sung jung,M,2,2,1,1,...,N,NaN,19990601,01,NaN,"토너, 잉크, 프린터 외 컴퓨터 관련 주변기기",NaN,NaN,이상호,1
3,2248114507,(주),새한실업,NaN,"SAEHAN INDUSTRY CO.,LTD.",M,2,2,1,1,...,N,NaN,19980522,01,NaN,금속 캔 및 기타 포장용기 제조업,NaN,NaN,김정남외 1명,6
4,1168132785,(주),석영엔터프라이즈,NaN,"Seokyoung Enterprise Co.,Ltd.",M,2,2,1,1,...,N,NaN,19930226,01,NaN,스마트트레이 외,NaN,www.systeel.co.kr,박성진,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,4808501629,NaN,게방식당팩토리,NaN,gebangsikdangfactory,M,2,2,1,2,...,N,8474403,20180126,01,NaN,간장게장(장류),NaN,https://gebangsikdang.modoo.at/,방건혁,NaN
6735,8018501578,(주),우진푸드중부지점,NaN,woojinfood,M,2,2,1,2,...,N,4033821,20200727,01,NaN,"돈가스, 탕수육, 꿔바로우",NaN,우진푸드.com,신남정,22
6736,4118523419,(주),윤월드푸드코리아다크써클브로스,NaN,"Youn World Food Korea Co., Ltd.",M,2,2,1,2,...,N,9189675,20190926,01,NaN,"한식, 중식",NaN,NaN,윤인수,0
6737,1738102533,(주),기린테크,NaN,"Girin Tech Co., Ltd.",M,2,2,1,1,...,N,NaN,20211025,01,NaN,1.인공지능(AI) 솔루션 구축 및 관련사업,NaN,NaN,임계현,NaN


In [6]:
#휴폐업 - 휴폐업이력 (6840개)
df_rest_record = pd.DataFrame(sheet_rest_record.values)
df_rest_record = df_rest_record.fillna('NaN')
df_rest_record.columns = df_rest_record.iloc[0,:]
df_rest_record = df_rest_record.iloc[1:, :]
df_rest_record = df_rest_record.reset_index(drop=True)
df_rest_record

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,1268145633,99991231,20220518,3,20220518
1,1388101749,99991231,20210319,3,20210319
2,1068179240,99991231,20200731,3,20191213
3,2248114507,99991231,20210531,3,20210531
4,1168132785,99991231,20210630,3,20210630
...,...,...,...,...,...
6835,4808501629,99991231,20211231,3,20211231
6836,8018501578,99991231,20211126,3,20211126
6837,4118523419,99991231,20220630,3,20220630
6838,1738102533,99991231,20220731,3,20220731


In [7]:
#액티브 기업
sheet_active = wb2['기업개요_외감']
sheet_active_rest = wb2['휴폐업이력']

In [8]:
#액티브 - 기업개요_외감
df_active= pd.DataFrame(sheet_active.values)
df_active.columns = df_active.iloc[0, :]
df_active = df_active.iloc[1:, :]
df_active = df_active.replace('','NaN')
df_active = df_active.fillna('NaN')
df_active = df_active.reset_index(drop=True)
df_active

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,...,N,NaN,19660419,01,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,...,N,NaN,19910806,01,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,...,N,NaN,20011030,01,NaN,시내버스업,NaN,transport@naver.com,김명화,149
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,...,N,NaN,19990112,01,NaN,자동차 부품 제조,NaN,www.koreajw.com,김명구,75
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,...,N,NaN,20000531,01,NaN,골프용품 전반의 제조 및 수출업,NaN,NaN,김홍득,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,NaN,제이앤인베스트먼트,NaN,J&INVESTMENT,M,2,2,1,1,...,N,NaN,20191206,01,NaN,경영컨설팅업,NaN,NaN,정경태,0
28978,8028701430,NaN,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,1,...,N,NaN,20200402,01,NaN,비주거용 건물 임대업,NaN,NaN,송성현,1
28979,4268501421,NaN,마스비 건축연구소,NaN,NaN,M,2,2,1,2,...,N,1220726,19900108,01,NaN,"BIM용역, 엔지니어링컨설팅",NaN,NaN,김태남,81
28980,6518502462,(주),건설레미콘의정부공장,NaN,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,2,...,N,1583116,20220615,01,NaN,레미콘,NaN,NaN,이미성,26


In [9]:
#액티브 - 휴폐업이력 (4828개)
df_active_record = pd.DataFrame(sheet_active_rest.values)
df_active_record.columns = df_active_record.iloc[0, :]
df_active_record = df_active_record.iloc[1:, :]
df_active_record = df_active_record.fillna('NaN')
df_active_record = df_active_record.reset_index(drop=True)
df_active_record

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN
...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN
4823,6098801857,99991231,20220921,6,NaN
4824,6098801857,20220920,20220607,3,20220607
4825,4988102183,99991231,20220705,5,NaN


## 데이터 전처리

In [10]:
#사용할 컬럼들
select_columns = ['BIZ_NO', 'BZ_TYP','NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'EMP_CNT']
len(select_columns)

19

In [11]:
df_rest_select = df_rest.loc[:, select_columns]
df_active_select = df_active.loc[:, select_columns]

#### 라벨인코딩 - FR_IVST_CORP_YN , VENT_YN, MDSCO_PRTC_YN, NATN_NM

In [12]:
encode_features = ['FR_IVST_CORP_YN' , 'VENT_YN', 'MDSCO_PRTC_YN', 'NATN_NM']

for df in [df_rest_select, df_active_select]:
    for feature in encode_features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])

df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1138111739,M,0,2,1,1,0,0,NaN,10,47712,NaN,NaN,0,NaN,19660419,01,NaN,43
1,2078132193,M,0,2,1,1,0,0,NaN,10,15190,NaN,NaN,0,NaN,19910806,01,NaN,35
2,1318167565,M,0,2,1,1,0,0,NaN,10,49212,NaN,NaN,0,NaN,20011030,01,NaN,149
3,5048128251,M,0,2,1,1,0,0,NaN,10,30391,NaN,NaN,0,NaN,19990112,01,NaN,75
4,1098164260,M,0,2,1,1,0,0,NaN,10,46800,NaN,NaN,0,NaN,20000531,01,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,20191206,01,NaN,0
28978,8028701430,M,0,2,1,1,0,0,NaN,10,68122,NaN,NaN,0,NaN,20200402,01,NaN,1
28979,4268501421,M,0,2,1,2,0,0,NaN,10,72100,NaN,NaN,0,1220726,19900108,01,NaN,81
28980,6518502462,M,0,2,1,2,0,0,NaN,10,23322,NaN,NaN,0,1583116,20220615,01,NaN,26


In [13]:
df_active_select['NATN_NM'].value_counts()

0     27337
45      379
20      276
33      146
3       104
      ...  
9         1
22        1
57        1
39        1
12        1
Name: NATN_NM, Length: 64, dtype: int64

In [14]:
#국가(NATN_NM) 64개 - null값이 27337이라 국가 그룹화 의미x?
countryDF = pd.DataFrame(df_active_select['NATN_NM'].value_counts())
country_list=countryDF.index.tolist()
len(country_list)

64

In [15]:
#PBCO_GB	공기업구분 1.공기업 2.일반기업
#PSN_CORP_GB	개인법인구분 1.법인 2.개인
#HDOF_BR_GB	본점지점구분 1.본점 2.지점

#하나의 컬럼마다 2차원데이터프레임으로 존재해야해서 원핫인코딩 불가
#라벨인코딩도 굳이 안해도 될 듯해서 일단 pass

#### 상장기업(LIST_CD) 처리

In [16]:
#LIST_CD 상장코드 - 상장유무로 0,1로 바꾸기

df_active_select['LIST_CD'] = df_active_select['LIST_CD'].replace('NaN',0)
df_rest_select['LIST_CD'] = df_rest_select['LIST_CD'].replace('NaN',0)

In [17]:
df_active_select['LIST_CD'].value_counts()

0          28893
A138290        1
A040470        1
A043500        1
A271780        1
           ...  
A151750        1
A028040        1
A102210        1
A087730        1
A340120        1
Name: LIST_CD, Length: 90, dtype: int64

In [18]:
df_active_select.loc[df_active_select['LIST_CD']!=0]=1
df_rest_select.loc[df_rest_select['LIST_CD']!=0]=1

In [19]:
#액티브 - 기업개요(외감) 상장기업 89개
df_active_select['LIST_CD'].value_counts()

0    28893
1       89
Name: LIST_CD, dtype: int64

In [20]:
#휴폐업 - 기업개요 상장기업 46개 / 비상장6693
df_rest_select['LIST_CD'].value_counts()

0    6693
1      46
Name: LIST_CD, dtype: int64

In [21]:
#상장기업인지 아닌지 - 컬럼 추가
df_active_select['Listing'] = df_active_select['LIST_CD']
df_rest_select['Listing'] = df_rest_select['LIST_CD']

#조건식을 따로 작성
listing_condition = (df_active_select['Listing'] != 0)
listing_condition2 = (df_rest_select['Listing'] != 0)

df_active_select.loc[listing_condition, 'Listing']=1
df_rest_select.loc[listing_condition2, 'Listing']=1

df_rest_select.head()

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing
0,1268145633,M,0,2,1,1,0,0,0,10,30391,NaN,NaN,0,NaN,20000529,01,NaN,40,0
1,1388101749,M,0,2,1,1,0,0,0,10,46610,NaN,NaN,0,NaN,19850110,01,NaN,13,0
2,1068179240,M,0,2,1,1,0,0,0,10,46510,NaN,NaN,0,NaN,19990601,01,NaN,1,0
3,2248114507,M,0,2,1,1,0,0,0,10,25991,NaN,NaN,0,NaN,19980522,01,NaN,6,0
4,1168132785,M,0,2,1,1,0,0,0,10,24199,NaN,NaN,0,NaN,19930226,01,NaN,40,0


In [22]:
df_rest_select.groupby('Listing').count()

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
Listing,,,,,,,,,,,,,,,,,,,
0,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693
1,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46


#### column 형변환

In [23]:
#type을 float로 바꿀 컬럼들
float_columns = ['NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'MDSCO_PRTC_YN', 'ESTB_GB']

#NaN값 있어서 못바꾸는 것들 EMP_CNT, IND_CD_ORDR
#직원수 NaN값 뭐할지 정하기 - 평균값?

In [24]:
#type 변경
for columns in float_columns:
    df_active_select[columns] = pd.to_numeric(df_active_select[columns])
    df_rest_select[columns] = pd.to_numeric(df_rest_select[columns])

#### 휴폐업이력 유무(close_y_n)와 휴폐업횟수(close_count) 컬럼 만들기

In [25]:
# 액티브-휴폐업이력 , 휴폐업-휴폐업이력 - CLSBZ_GB(휴폐업구분)컬럼을 int로 변경
df_active_record['CLSBZ_GB'] = pd.to_numeric(df_active_record['CLSBZ_GB'])
df_rest_record['CLSBZ_GB'] = pd.to_numeric(df_rest_record['CLSBZ_GB'])

In [26]:
#close_y_n : 휴폐업 유무
df_rest_record['close_y_n']=1
df_active_record['close_y_n']=1

In [27]:
#BIZ_NO로 set인덱스
df_active_record=df_active_record.set_index('BIZ_NO')
df_rest_record=df_rest_record.set_index('BIZ_NO')

In [28]:
#휴폐업 조건으로 필터링한 새로운 컬럼 - 휴폐업이력 있는 기업만 들어있는 df
close_condition = (df_active_record.CLSBZ_GB==1) | (df_active_record.CLSBZ_GB==2) | (df_active_record.CLSBZ_GB==3)
close_condition2 = (df_rest_record.CLSBZ_GB==1) | (df_rest_record.CLSBZ_GB==2) | (df_rest_record.CLSBZ_GB==3)
condition_df_active_record = df_active_record[close_condition]
condition_df_rest_record = df_rest_record[close_condition2]

In [29]:
#휴폐업 횟수 구하기
condition_df_active_record = condition_df_active_record.groupby('BIZ_NO').sum()
condition_df_rest_record = condition_df_rest_record.groupby('BIZ_NO').sum()

In [30]:
# 휴폐업 횟수 컬럼 추가
df_active_record['close_count'] = condition_df_active_record['close_y_n']
df_rest_record['close_count'] = condition_df_rest_record['close_y_n']

In [31]:
#index reset
df_active_record=df_active_record.reset_index()
df_rest_record=df_rest_record.reset_index()

In [32]:
#BIZ_NO 형변환
df_active_record['BIZ_NO'] = pd.to_numeric(df_active_record['BIZ_NO'])
df_rest_record['BIZ_NO'] = pd.to_numeric(df_rest_record['BIZ_NO'])

In [33]:
#중복인덱스 삭제하기 - 열지정하여 중복제거
df_active_record= df_active_record.drop_duplicates(['BIZ_NO']).reset_index(drop=True)
df_rest_record= df_rest_record.drop_duplicates(['BIZ_NO']).reset_index(drop=True)

#### 업력만들기
- df_active_select, df_rest_select 에 업력 컬럼추가하고
- 사업자번호 기준으로 위의 df랑 df_active_record, df_rest_record 결합하기


- 지금 안되는 이유
- ESTB_DATE에 null값이 있어서 NaN으로 채워줌 
- pd.to_datetime으로 변환하려고 하니 NaN은 int가 아니라서 오류
- 나머지 날짜들도 str형식이라 pd.numeric으로 int로 바꿔준다음 pd.to_datetime해줘야하는데,  중간중간에 NaN이 있으니까 오류
- try, except로 예외처리도 안됨 -> 전부 except로 넘어가버림.. 

test
-> null값을 99999999로 채워보기

In [94]:
test = pd.DataFrame(sheet_active.values)
test.columns = test.iloc[0, :]
test = test.iloc[1:, :]
test = test.reset_index(drop=True)
test=test.replace('', '99999999')
test = test.fillna('99999999')

In [95]:
test = test['ESTB_DATE']
test=pd.DataFrame(test.values)
test.columns=['estb_date']

In [96]:
#null값의 개수 1074
test.loc[test['estb_date']=='99999999']

,estb_date
3327,99999999
11415,99999999
11958,99999999
11959,99999999
11965,99999999
...,...
27100,99999999
27663,99999999
27715,99999999
27875,99999999


In [72]:
# test['estb_date'] = pd.to_numeric(test['estb_date'])
# test['estb_date'] = test['estb_date'].astype('int')

In [98]:
type(test['estb_date'][0])

str

In [99]:
type(test['estb_date'][28534])

str

In [103]:
#20000700 같이 날짜가 이상한 값 수정하기(00을 01로 바꿔주기)
#str슬라이싱으로 4,2,2, 사이에 . (2022.10.16 형식이어야 pd.to_datetime이 오류없을까 싶어서 해보는 중)
test['date_dot'] = test['estb_date'].str[:4]+'.'+test['estb_date'].str[4:6]+'.'+test['estb_date'].str[6:]
test

,estb_date,date_dot
0,19660419,1966.04.19
1,19910806,1991.08.06
2,20011030,2001.10.30
3,19990112,1999.01.12
4,20000531,2000.05.31
...,...,...
28977,20191206,2019.12.06
28978,20200402,2020.04.02
28979,19900108,1990.01.08
28980,20220615,2022.06.15


In [122]:
#숫자 비교해서 조건줘야하니까 일단 type int로 변경
test['day'] = test['estb_date'].str[6:].astype('int')

In [131]:
#day 조건 따로 줘서 해당하면 값 바꾸기 - day가 1~31의 값이 아니면 1주기
day_condiiton = (test['day']<1) | (test['day']>31)
test.loc[day_condiiton, 'day']=1

In [134]:
test.loc[test['estb_date']=='20000700']

,estb_date,date_dot,day
638,20000700,2000.07.00,1


In [141]:
test['day'] = test['day'].astype('str')

In [144]:
test.dtypes

estb_date    object
day          object
dtype: object

estb_date    object
day          object
dtype: object

In [150]:
#day숫자들 전부 두글자로 바꿔주기
test['day'] = test['day'].format()
# pandas로 두글자로 ㅂ꾸는 법?



TypeError: %d format: a number is required, not Series

In [145]:
test['real_date'] = test['estb_date'].str[:4] + '.' + test['estb_date'].str[4:6] + '.' + test['day']

AttributeError: 'Series' object has no attribute 'zfill'

In [39]:
# test['estb_datetime'] = pd.to_datetime(test['estb_date'],format='%Y%m%d').dt.strftime('%Y-%m-%d')

In [107]:
#오류 20000700 이라서 남.. 
try:
    test['estb_datetime'] = pd.to_datetime(test['date_dot'])
except:
    pass

In [108]:
test

,estb_date,date_dot
0,19660419,1966.04.19
1,19910806,1991.08.06
2,20011030,2001.10.30
3,19990112,1999.01.12
4,20000531,2000.05.31
...,...,...
28977,20191206,2019.12.06
28978,20200402,2020.04.02
28979,19900108,1990.01.08
28980,20220615,2022.06.15


In [42]:
try:
    test['datetime'] = pd.to_datetime(test['date'])
except:
    test['datetime']=test['date']


#값중에서 이상한 값이 있으면 pd로 적용이 안되나봐, 이건 이상치가 하나라도 있으면 적용이 안되는듯?
#그래서 전부 예외처리를 해버리는 것지...
#이상치 = '20000700' 이나 fillna처리한 '99999999'

KeyError: 'date'

In [ ]:
#df_active_select, df_rest_select에 업력 컬럼 만들기
#업력=설립날짜-오늘날짜   (휴업기간는 고려x) ESTB_DATE




# history = (now-estb_date)
# his_day = history.days
# his_year= (his_day)//365


#### 구간화
- 직원수EMP_CNT	, 설립일자ESTB_DATE (업력)

#### 그룹화? 
- 업종, 나라?  - BZ_TYP, IND_CD1	

#### 데이터프레임 join (BIZ_NO기준으로)
- df_active_select + df_active_record
- df_rest_select + df_rest_record

In [ ]:
df_active_select2 = df_active_select.copy()
df_active_select2['BIZ_NO'] = pd.to_numeric(df_active_select2['BIZ_NO'])
df_active_record2 = df_active_record.copy()

In [ ]:
df_rest_select2 = df_rest_select.copy()
df_rest_select2['BIZ_NO'] = pd.to_numeric(df_rest_select2['BIZ_NO'])
df_rest_record2 = df_rest_record.copy()

In [ ]:

#28982+2083-362(교집합)= 30703
# 28982 컬럼에 362개만 휴폐업횟수컬럼만 넣고 싶음

#df_rest_select2 (6739), df_rest_record2 (6746) -> 아우터조인 6792

In [ ]:
#액티브 교집합(inner join) (362개)
df_active_innerjoin = pd.merge(df_active_select2, df_active_record2,left_on='BIZ_NO',right_on='BIZ_NO', how='inner')
df_active_innerjoin = df_active_innerjoin[['BIZ_NO', 'close_y_n', 'close_count']]


In [ ]:
#휴폐업 기업개요와 휴폐업이력의 교집합(6693)
df_rest_innerjoin = pd.merge(df_rest_select2, df_rest_record2,left_on='BIZ_NO',right_on='BIZ_NO', how='inner')
df_rest_innerjoin = df_rest_innerjoin[['BIZ_NO', 'close_y_n', 'close_count']]
df_rest_innerjoin

#휴폐업 기업개요(6739=유니크), 휴폐업이력(6840)(유니크6746)

,BIZ_NO,close_y_n,close_count
0,1268145633,1,1
1,1388101749,1,1
2,1068179240,1,1
3,2248114507,1,1
4,1168132785,1,1
...,...,...,...
6688,4808501629,1,1
6689,8018501578,1,1
6690,4118523419,1,1
6691,1738102533,1,1


In [ ]:
#outer join하니까 기존 select에 + record 합해지고 그그그ㅡ 이력 겹치는 362개는 기존에 있던 컬럼에 합쳐지네 .. 그러니가
# #28982+2083-362(교집합)= 30703 이 나오는거지.... 

# 교집합만 따로 df만들어서 select에 outer join하면 될듯

In [ ]:
#위에 꺼 하기 - 액티브 기업외감 df에 휴폐업 이력이 있는 것듬만 합치기 완료
df_active_join = pd.merge(df_active_select2, df_active_innerjoin ,left_on='BIZ_NO',right_on='BIZ_NO', how='outer')
df_rest_join = pd.merge(df_rest_select2, df_rest_innerjoin ,left_on='BIZ_NO',right_on='BIZ_NO', how='outer')
df_rest_join


,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,...,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing,close_y_n,close_count
0,1268145633,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20000529,1,NaN,40,0,1.0,1.0
1,1388101749,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19850110,1,NaN,13,0,1.0,1.0
2,1068179240,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19990601,1,NaN,1,0,1.0,1.0
3,2248114507,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19980522,1,NaN,6,0,1.0,1.0
4,1168132785,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19930226,1,NaN,40,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,4808501629,M,0,2,1,2,0,0,0,10,...,NaN,0,8474403,20180126,1,NaN,NaN,0,1.0,1.0
6735,8018501578,M,0,2,1,2,0,0,0,10,...,NaN,0,4033821,20200727,1,NaN,22,0,1.0,1.0
6736,4118523419,M,0,2,1,2,0,0,0,10,...,NaN,0,9189675,20190926,1,NaN,0,0,1.0,1.0
6737,1738102533,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20211025,1,NaN,NaN,0,1.0,1.0


In [ ]:
#휴폐업 기업개요와 휴폐업이력의 교집합(6693) 전부 1값(당연함)
df_rest_join['close_y_n'].value_counts()

1.0    6693
Name: close_y_n, dtype: int64

In [ ]:
df_rest_join['close_count'].value_counts()

1.0    6604
2.0      88
4.0       1
Name: close_count, dtype: int64

##### df_active_join, df_rest_join에 생긴 null값 채우기

In [ ]:
#rest_join의 close_y_n랑 close_count에 null값이 46개 생김 ,  교집합이 6693이고 널값이 46 => 합하면 6739(join값)
df_rest_join

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,...,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing,close_y_n,close_count
0,1268145633,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20000529,1,NaN,40,0,1.0,1.0
1,1388101749,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19850110,1,NaN,13,0,1.0,1.0
2,1068179240,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19990601,1,NaN,1,0,1.0,1.0
3,2248114507,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19980522,1,NaN,6,0,1.0,1.0
4,1168132785,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19930226,1,NaN,40,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,4808501629,M,0,2,1,2,0,0,0,10,...,NaN,0,8474403,20180126,1,NaN,NaN,0,1.0,1.0
6735,8018501578,M,0,2,1,2,0,0,0,10,...,NaN,0,4033821,20200727,1,NaN,22,0,1.0,1.0
6736,4118523419,M,0,2,1,2,0,0,0,10,...,NaN,0,9189675,20190926,1,NaN,0,0,1.0,1.0
6737,1738102533,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20211025,1,NaN,NaN,0,1.0,1.0


In [ ]:
#일단 휴폐업join의 널값을 NaN으로 채워주겟음
df_rest_join = df_rest_join.fillna('NaN')

In [ ]:
#df_active_join에서 close_y_n, close_count 가 NaN값이면 0으로 바꿔주기
NaN_condition = (df_active_join['close_y_n']!=1)
df_active_join.loc[NaN_condition, 'close_y_n']=0
df_active_join



,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,...,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing,close_y_n,close_count
0,1138111739,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19660419,1,NaN,43,0,0.0,NaN
1,2078132193,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19910806,1,NaN,35,0,0.0,NaN
2,1318167565,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20011030,1,NaN,149,0,0.0,NaN
3,5048128251,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,19990112,1,NaN,75,0,0.0,NaN
4,1098164260,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20000531,1,NaN,3,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,0,NaN,...,NaN,0,NaN,20191206,1,NaN,0,0,0.0,NaN
28978,8028701430,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20200402,1,NaN,1,0,0.0,NaN
28979,4268501421,M,0,2,1,2,0,0,0,10,...,NaN,0,1220726,19900108,1,NaN,81,0,0.0,NaN
28980,6518502462,M,0,2,1,2,0,0,0,10,...,NaN,0,1583116,20220615,1,NaN,26,0,0.0,NaN


In [ ]:
#df_active_join에서 close_count 가 NaN값이면 0으로 바꿔주기


df_active_join= df_active_join.fillna(0)
df_active_join['close_count'].value_counts()

#되긴했는데 이상한 점, 분명 맨 처음에 null 값을 NaN으로 다 채워줘서 isnull이 0인거 확인했는데
#copy하고 새로 만들어서 그런가?(근데 그것도 이상하지 카피인데..) close_count의 NaN이 결측치로 처리된다. -> 그래서 fillna로 채워줌!

0.0    28629
1.0      301
2.0       35
3.0       11
4.0        4
7.0        1
5.0        1
Name: close_count, dtype: int64

In [ ]:
#확인
df_active_join.loc[df_active_join['BIZ_NO']==5768700468]

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,...,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing,close_y_n,close_count
25835,5768700468,M,0,2,1,1,0,0,0,10,...,NaN,0,NaN,20130909,1,NaN,310,0,1.0,1.0


데이터 전처리(라벨인코딩)
컬럼 만들기 - 휴폐업유무(0,1), 휴폐업횟수(close_count)
(정답y = 휴폐업 이력 유무)
액티브재무보유 - 기업개요(외감), 휴페업이력
휴폐업재무보유 - 기업개요, 휴폐업이력

x값들이 필요
#휴폐업 이력이 있는, 현재 액티브리스트에 있는 회사들 - 362개
#휴폐업재무정보에서 휴폐업이력 사업자번호와 겹치는거 6739 (휴폐업재무정보는 6746개)
df를 합쳐서 기업별로(사업자코드별로) (휴폐업이력sheet에 있는) 휴폐업이력 유무와 휴폐업 횟수를 알아야하는 것. 
